# Emotion Prediction from RGB Data Using CNN

This Jupyter Notebook demonstrates how to predict emotions from RGB data using a Convolutional Neural Network (CNN) in Keras. We will use this demmo data generated to simulate RGB values associated with different emotional states.

## Table of Contents
1. Importing Libraries
2. Loading and Preparing the Data
3. Data Preprocessing
4. Building the CNN Model
5. Training the Model
6. Evaluating the Model
7. Conclusion and Further Steps

## 1. Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.utils import to_categorical

## 2. Loading and Preparing the Data

In [3]:
# Load the data
df = pd.read_csv('demmo_data.csv')
print(df.head())

# Check the number of unique emotional words
print("Unique emotional words:", df['Emotional_Word'].nunique())

df.head()

  Emotional_Word            RGB_1            RGB_2            RGB_3
0           Cute  [251, 167, 157]  [255, 242, 124]    [179, 22, 61]
1      Childlike   [251, 103, 89]  [255, 242, 124]  [153, 216, 212]
2         Pretty  [251, 167, 157]   [255, 242, 63]   [78, 181, 135]
3          Sweet   [251, 103, 89]  [253, 192, 145]  [251, 174, 193]
4        Amusing   [253, 166, 74]   [140, 201, 25]   [90, 177, 132]
Unique emotional words: 77


,Emotional_Word,RGB_1,RGB_2,RGB_3
0,Cute,"[251, 167, 157]","[255, 242, 124]","[179, 22, 61]"
1,Childlike,"[251, 103, 89]","[255, 242, 124]","[153, 216, 212]"
2,Pretty,"[251, 167, 157]","[255, 242, 63]","[78, 181, 135]"
3,Sweet,"[251, 103, 89]","[253, 192, 145]","[251, 174, 193]"
4,Amusing,"[253, 166, 74]","[140, 201, 25]","[90, 177, 132]"


## 3. Data Preprocessing
##### Convert the RGB values and emotional words to a suitable format for training a CNN.

In [9]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.utils import to_categorical

# Combine the RGB values into a single array and normalize them
X = np.array(df[['RGB_1', 'RGB_2', 'RGB_3']].apply(lambda x: np.array([eval(x['RGB_1']), eval(x['RGB_2']), eval(x['RGB_3'])]), axis=1).tolist())
X = X / 255.0  # Normalize RGB values

# Encode the emotional words
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Emotional_Word'])
y = to_categorical(y)

# Split the data into training, validation, and test sets
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training data shape:", X_train.shape, y_train.shape)
print("Validation data shape:", X_val.shape, y_val.shape)
print("Test data shape:", X_test.shape, y_test.shape)

Training data shape: (53, 3, 3) (53, 77)
Validation data shape: (12, 3, 3) (12, 77)
Test data shape: (12, 3, 3) (12, 77)


In [8]:
print("First 5 elements of X_train:", X_train[:5])
print("First 5 elements of y_train:", y_train[:5])

print("First 5 elements of X_val:", X_val[:5])
print("First 5 elements of y_val:", y_val[:5])

print("First 5 elements of X_test:", X_test[:5])
print("First 5 elements of y_test:", y_test[:5])

First 5 elements of X_train: [[[0.54901961 0.78823529 0.09803922]
  [0.20392157 0.61176471 0.3254902 ]
  [0.30588235 0.70980392 0.52941176]]

 [[0.89019608 0.10196078 0.16470588]
  [0.20784314 0.1254902  0.02352941]
  [0.12941176 0.14117647 0.03921569]]

 [[0.89019608 0.10196078 0.16470588]
  [0.37254902 0.1372549  0.55294118]
  [0.97254902 0.45882353 0.61568627]]

 [[0.98431373 0.40392157 0.34901961]
  [0.99215686 0.75294118 0.56862745]
  [1.         0.94901961 0.24705882]]

 [[0.12941176 0.14117647 0.03921569]
  [0.38431373 0.45490196 0.44705882]
  [0.01960784 0.04313725 0.58039216]]]
First 5 elements of y_train: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

## 4. Building the CNN Model
##### I use a VGG-16-like architecture adapted for my dataset

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential()

# Convolutional layers
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(3, 3, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu' , padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Fully connected layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.summary()


c:\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 1, 1, 64)       │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 0, 0, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 0, 0, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 0, 0, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 0, 0, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 0, 0, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 0)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 77)             │        39,501 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 115,661 (451.80 KB)

 Trainable params: 115,661 (451.80 KB)

 Non-trainable params: 0 (0.00 B)

## 5. Training the Model


In [12]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=32, 
                    validation_data=(X_val, y_val))


Epoch 1/50


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 3, 3), dtype=float32). Expected shape (None, 3, 3, 3), but input has incompatible shape (None, 3, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 3, 3), dtype=float32)
  • training=True
  • mask=None

## 6. Evaluating the Model

In [11]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Making predictions
def predict_emotion(rgb_combination):
    rgb_combination = np.array(rgb_combination).reshape((1, 3, 3, 1))
    prediction = model.predict(rgb_combination)
    return label_encoder.inverse_transform([np.argmax(prediction)])[0]

# Example prediction
sample_rgb_combination = [[251/255.0, 167/255.0, 157/255.0], [255/255.0, 242/255.0, 124/255.0], [179/255.0, 22/255.0, 61/255.0]]
print(predict_emotion(sample_rgb_combination))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.0000e+00 - loss: 6.3176
Test Accuracy: 0.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Pretty


## 7. Conclusion and Further Steps
